In [1]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from PIL import Image

os.getcwd()

'/home/matthias/monkey/MC-FairMOT_mt/own_ideas'

## Converting VoTT output

After labeling with VoTT, we obtain a .csv file, which we convert into many .txt files, one file for every frame. These will be saved in a folder called labels_with_ids (if it does not exist you have to create it.

In [327]:
data_root = "/usr/users/vogg/Labelling/Baboons/"

In [328]:
labels = pd.read_csv(data_root + "images/Baboon-finder-export.csv")

In [329]:
labels.head()

,image,xmin,ymin,xmax,ymax,label
0,IMAG0043%20-%202015%20-%20jan%20-%20ct3.JPG,352.207792,292.987013,574.545455,610.909091,baboon
1,IMAG0043%20-%202015%20-%20jan%20-%20ct3.JPG,1.038961,280.519481,437.402597,718.961039,baboon
2,IMAG0043%20-%202015%20-%20jan%20-%20ct3.JPG,929.870130,480.000000,1220.779221,700.259740,baboon
3,IMAG0043%20-%202015%20-%20jan%20-%20ct3.JPG,545.454545,309.610390,1204.155844,775.064935,baboon
4,IMAG0043%20-%202015%20-%20jan%20-%20ct3.JPG,1285.194805,658.701299,1628.051948,955.844156,baboon


In [330]:
img_list = os.listdir(data_root + "images")
regex = re.compile('.*JPG')
img_list = sorted([i for i in img_list if regex.match(i)])

In [331]:
img_id = img_list[1]

img = np.asarray(Image.open(data_root + "images/" + img_id))
#fig, ax = plt.subplots(figsize=(640/96, 640/96), dpi=96)
#ax.imshow(img)
img_h, img_w, _ = img.shape
img_h

1920

In [332]:
i_monkey = 0

for img_id in img_list:
    idx = np.where(labels.image == img_id)
    image_labels = labels.iloc[idx]
    
    label_fpath = data_root + "labels_with_ids/" + re.sub(".JPG", ".txt", img_id)

    img = np.asarray(Image.open(data_root + "images/" + img_id))
    img_h, img_w, _ = img.shape

    for index, row in image_labels.iterrows():

        #if row.label == "baboon":
        x_center = row.xmin + (row.xmax - row.xmin)/2
        y_center = row.ymin + (row.ymax - row.ymin)/2
        w = row.xmax - row.xmin
        h = row.ymax - row.ymin

        #i_monkey = row.label

        img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_monkey += 1
        with open(label_fpath, 'a') as f:
            f.write(label_str)


            

### Converting CVAT output

Data comes in separate files.

In [353]:
video_id = 'VID_20210227_133440_0'

data_root = "/usr/users/agecker/datasets/macaque_videos/"

path_o = data_root + video_id + "/obj_train_data/"

frame_list = os.listdir(path_o)
frame_list.sort()

In [354]:
regex = re.compile('.*txt')
frame_list = [i for i in frame_list if regex.match(i)]
print(len(frame_list))

361


In [343]:

!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/

mkdir: cannot create directory ‘/usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/’: File exists


In [356]:
!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210227_133440_0/images/

In [357]:
for frame_id in frame_list:
    text = pd.read_csv(path_o + frame_id, names = [1,2,3,4,5], sep = " ")
    text.insert(0, "z", np.zeros(len(text)))
    text.z = text.z.astype(int)
    text.to_csv(data_root + video_id + "/labels_with_ids/" + frame_id,
                 sep=' ', index=False, header = False)

## Videos to frames

In [358]:
import cv2
import sys
#sys.path

In [359]:
path = "/usr/users/agecker/datasets/macaque_videos/"
cap = cv2.VideoCapture(path + "Videos/" + video_id + ".mp4")

#Total number of frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)

count = 0

Frame count: 361


In [360]:
#%%

while count < frame_count:
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
    success, image = cap.read()
    cv2.imwrite(path + video_id + "/images/frame_%s.jpg" % str(count).zfill(6), image)     # save frame as JPEG file
    count += 1
    if count % 50 == 0:
        print(count)

50
100
150
200
250
300
350


## All files for training list - Wildmacaques

In [361]:
path = "/usr/users/agecker/datasets/macaque_videos/"

vids = os.listdir(path)

regex = re.compile('VID_.*')
vids = [i for i in vids if regex.match(i)]
vids

['VID_20210223_123630_0',
 'VID_20210227_133440_0',
 'VID_20210224_114038_0',
 'VID_20210224_115729_0',
 'VID_20210223_123817_0',
 'VID_20210228_153846_0',
 'VID_20210224_115455_0',
 'VID_20210223_123854_0',
 'VID_20210227_133251_0']

In [362]:
for i, out_vid in enumerate(vids):
    temp_vids = [x for x in vids if x != out_vid]
    
    for vid in temp_vids:
        img_ids = os.listdir(path + vid + "/images")


        label_fpath = "/usr/users/vogg/FairMOT/src/data/wildmacaques" + str(i) + ".train"
        regex = re.compile("frame.*")
        for img_id in sorted(img_ids):

            if regex.match(img_id):
                label_str =  "macaque_videos/" + vid + "/images/" + img_id + "\n"

                with open(label_fpath, 'a') as f:
                    f.write(label_str)

## Make files for training list images

In [272]:
for img_id in img_list:
    label_fpath = "/usr/users/vogg/FairMOT/src/data/baboons.train"

    label_str =  "Baboons/images/" + img_id + "\n"

    with open(label_fpath, 'a') as f:
        f.write(label_str)